In [206]:
A = [
    [1.15, 0.42, 10.10, 4.25],
    [1.59, 0.55, -0.32, 0.29],
    [1.14, 3.15, 2.05, 7.86],
    [0.77, 6.11, -3.01, 0.74]
]

B = [
    [15.08],
    [1.01],
    [7.90],
    [-7.61]
]

eps = 1e-7
ndigits = 8

In [207]:
def gauss_get_expanded_matrix(a: list[list[float]], b: list[list[float]]) -> list[list[float]]:
    expanded_matrix = []
    for i in range(len(a)):
        a[i].append(b[i][0])
        expanded_matrix.append(a[i])
    return expanded_matrix

In [208]:
def gauss_choose_main_element(matrix: list[list[float]], step: int = 0) -> None:
    main_element_index = step
    for row_index in range(step, len(matrix)):
        if abs(matrix[row_index][step]) > abs(matrix[main_element_index][step]):
            main_element_index = row_index
    matrix[step], matrix[main_element_index] = matrix[main_element_index], matrix[step]

In [209]:
def gauss_get_diagonal_matrix(expanded_matrix: list[list[float]], ndigits: int=8) -> None:
    for step in range(len(expanded_matrix) - 1):
        gauss_choose_main_element(expanded_matrix, step)
        pivot = expanded_matrix[step][step]
        if abs(pivot) < 1e-12:
            raise ValueError("Система несовместна или имеет бесконечно много решений")
        for row_index in range(step + 1, len(expanded_matrix)):
            factor = expanded_matrix[row_index][step] / pivot
            for col_index in range(step, len(expanded_matrix[row_index])):
                expanded_matrix[row_index][col_index] = round(expanded_matrix[row_index][col_index] -
                                                         expanded_matrix[step][col_index] * factor, ndigits)
                if abs(expanded_matrix[row_index][col_index]) < 1e-12:
                    expanded_matrix[row_index][col_index] = 0.0

In [210]:
def express_x(x: list[float], matrix: list[list[float]], ndigits: int = 8) -> list[float]:
    if len(x) != len(matrix):
        raise "Невозможно выразить x - размерности матриц не совпадают"
    for step in range(len(matrix) - 1, -1, -1):
        x_step = round(
            (expanded_matrix[step][-1] -
             sum([matrix[step][i] * x[i] for i in range(len(matrix[step]) - 1) if i != step]))/ matrix[step][step], ndigits)
        x[step] = x_step
    return x

In [211]:
def gauss_get_solve(expanded_matrix: list[list[float]], ndigits: int=8) -> list[float]:
    solved_matrix = [0] * len(expanded_matrix)
    return express_x(solved_matrix, expanded_matrix, ndigits)

In [212]:
a, b = [row.copy() for row in A], [row.copy() for row in B]

expanded_matrix = gauss_get_expanded_matrix(a, b)
gauss_get_diagonal_matrix(expanded_matrix, ndigits)
print('Матрица треугольного вида')
for row in expanded_matrix:
    print(row)

x_gauss = gauss_get_solve(expanded_matrix, ndigits)
print('Решение методом Гаусса: ', x_gauss)

Матрица треугольного вида
[1.59, 0.55, -0.32, 0.29, 1.01]
[0.0, 5.8436478, -2.85503145, 0.59955975, -8.0991195]
[0.0, 0.0, 10.34229341, 4.03797372, 14.38026714]
[0.0, 0.0, 0.0, 5.95372448, 5.9537245]
Решение методом Гаусса:  [1.0, -1.0, 1.0, 1.0]


In [213]:
def make_diagonally_dominant(matrix: list[list[float]]) -> None:
    for step in range(len(matrix) - 1):
        gauss_choose_main_element(matrix, step)

In [214]:
def seidel_check_converge(x: list[float], y: list[float]) -> bool:
    for i in range(len(x)):
        if abs(x[i] - y[i]) >= eps:
            return False
    return True

In [215]:
def seidel_find_solution_recursive(x: list[float], expanded_matrix: list[list[float]], ndigits: int = 8, step: int = 0) -> (list[float], int):
    x_old = x.copy()
    x_new = express_x(x, expanded_matrix, ndigits)
    if seidel_check_converge(x_new, x_old):
        return x_new, step
    return seidel_find_solution_recursive(x_new, expanded_matrix, ndigits, step + 1)

In [216]:
x0 = [0] * len(expanded_matrix)

a, b = [row.copy() for row in A], [row.copy() for row in B]
expanded_matrix = gauss_get_expanded_matrix(a, b)
make_diagonally_dominant(expanded_matrix)
print('Матрица с максимальными элементами на диагонали')
for row in expanded_matrix:
    print(row)

Матрица с максимальными элементами на диагонали
[1.59, 0.55, -0.32, 0.29, 1.01]
[0.77, 6.11, -3.01, 0.74, -7.61]
[1.15, 0.42, 10.1, 4.25, 15.08]
[1.14, 3.15, 2.05, 7.86, 7.9]


In [217]:
x_seidel, iteration_count = seidel_find_solution_recursive(x0, expanded_matrix, ndigits)
print('Решение методом Зейделя: ', x_seidel)

Решение методом Зейделя:  [1.0, -1.00000001, 0.99999999, 1.00000003]


In [218]:
import numpy as np

x_exact = [1.0, -1.0, 1.0, 1.0]

err_abs_gauss = np.max(np.abs(np.array(x_gauss) - np.array(x_exact)))
err_abs_zeidel = np.max(np.abs(np.array(x_seidel) - np.array(x_exact)))

print("Абсолютная погрешность по Гауссу:", err_abs_gauss)
print("Абсолютная погрешность по Зейделю:", err_abs_zeidel)

Абсолютная погрешность по Гауссу: 0.0
Абсолютная погрешность по Зейделю: 3.0000000039720476e-08
